In [6]:
import numpy as np
import tensorflow as tf

In [8]:
#Hyperparameters

hop=192               #hop size (window size = 6*hop)
sr=16000              #sampling rate
min_level_db=-100     #reference values to normalize data
ref_level_db=20

shape=24              #length of time axis of split specrograms to feed to generator            
vec_len=128           #length of vector generated by siamese vector
# bs = 16               #batch size
bs = 32               #batch size
delta = 2.            #constant for siamese loss

In [1]:


brecs = np.load('/data/wav/np/v1/bdata.npz', allow_pickle=True)
bdata = brecs['bdata']
bdata.shape

(34876, 192, 240, 1)

In [2]:
bspec = brecs['bspec']
bspec.shape

(7827,)

In [3]:
arecs = np.load('/data/wav/np/v1/adata.npz', allow_pickle=True)
adata = arecs['adata']
adata.shape

(39608, 192, 240, 1)

In [4]:
aspec = arecs['aspec']
aspec.shape

(7136,)

In [9]:
#Creating Tensorflow Datasets

@tf.function
def proc(x):
    return tf.image.random_crop(x, size=[hop, 3*shape, 1])

dsa = tf.data.Dataset.from_tensor_slices(adata).repeat(50).map(proc, num_parallel_calls=tf.data.experimental.AUTOTUNE).shuffle(10000).batch(bs, drop_remainder=True)
dsb = tf.data.Dataset.from_tensor_slices(bdata).repeat(50).map(proc, num_parallel_calls=tf.data.experimental.AUTOTUNE).shuffle(10000).batch(bs, drop_remainder=True)

In [ ]:
#Helper functions

#Generate spectrograms from waveform array
def tospec(data):
    specs=np.empty(data.shape[0], dtype=object)
    for i in range(data.shape[0]):
        x = data[i]
        S=prep(x)
        S = np.array(S, dtype=np.float32)
        specs[i]=np.expand_dims(S, -1)
    print(specs.shape)
    return specs

#Generate multiple spectrograms with a determined length from single wav file
def tospeclong(path, length=4*16000):
    x, sr = librosa.load(path,sr=16000)
    x,_ = librosa.effects.trim(x)
    loudls = librosa.effects.split(x, top_db=50)
    xls = np.array([])
    for interv in loudls:
        xls = np.concatenate((xls,x[interv[0]:interv[1]]))
    x = xls
    num = x.shape[0]//length
    specs=np.empty(num, dtype=object)
    for i in range(num-1):
        a = x[i*length:(i+1)*length]
        S = prep(a)
        S = np.array(S, dtype=np.float32)
        try:
            sh = S.shape
            specs[i]=S
        except AttributeError:
            print('spectrogram failed')
    print(specs.shape)
    return specs

#Waveform array from path of folder containing wav files
def audio_array(path):
    ls = glob(f'{path}/*.wav')
    adata = []
    for i in range(len(ls)):
        x, sr = tf.audio.decode_wav(tf.io.read_file(ls[i]), 1)
        x = np.array(x, dtype=np.float32)
        adata.append(x)
    return np.array(adata)

#Concatenate spectrograms in array along the time axis
def testass(a):
    but=False
    con = np.array([])
    nim = a.shape[0]
    for i in range(nim):
        im = a[i]
        im = np.squeeze(im)
        if not but:
            con=im
            but=True
        else:
            con = np.concatenate((con,im), axis=1)
    return np.squeeze(con)

# NB shape=24 #length of time axis of split specrograms to feed to generator   
#Split spectrograms in chunks with equal size
def splitcut(data, debug=False):
    ls = []
    mini = data[0].shape[1]
    minifinal = 10*shape                                                              #max spectrogram length
    for i in range(data.shape[0]-1):
        if data[i].shape[1]<=mini:
            mini = data[i].shape[1]
    
    if mini >= 3*shape and mini < minifinal:
        minifinal = mini
        
    if debug:
        print(f"Mininum chunk size is {minifinal}")
    
    for i in range(data.shape[0]):
        x = data[i]
        if debug:
            print(f"{i} {x.shape}-> {x.shape[1]} >= {3*shape}?")
            
        if x.shape[1]>=3*shape:
            if debug:
                print(f" Attempt to break the file up into {x.shape[1]//minifinal} chunks of size {minifinal}")
            
            for n in range(x.shape[1]//minifinal):
                ls.append(x[:,n*minifinal:n*minifinal+minifinal,:])
        ls.append(x[:,-minifinal:,:])
    
    return np.array(ls)

def splitcut_v2(data, debug=False):
    ls = []
    min_frames = 0
    # get the smallest 
    for i in range(data.shape[0]):
        min_frames = min(min_frames, data[i])
    
    # Ensure that the chunk_size is not larger than the max expected spectrogram length
    chunk_size = min(min_frames, 10*shape)
    
    for i in range(data.shape[0]):
        spectrogram  = data[i]
        if debug:
            print(f"{i} {spectrogram.shape}-> {spectrogram.shape[1]} >= {chunk_size}?")
        spectrogram_splits = [spectrogram[i * chunk_size: (i+1) * chunk_size] for i in range(int(np.ceil(len(spectrogram) / chunk_size)))]
        
        ls.append(spectrogram_splits)
    return np.array(ls)

In [11]:
#Adding Spectral Normalization to convolutional layers

from tensorflow.python.keras.utils import conv_utils
from tensorflow.python.ops import array_ops
from tensorflow.python.ops import math_ops
from tensorflow.python.ops import sparse_ops
from tensorflow.python.ops import gen_math_ops
from tensorflow.python.ops import standard_ops
from tensorflow.python.eager import context
from tensorflow.python.framework import tensor_shape
from tensorflow.keras.layers import Input, Dense, Reshape, Flatten, Concatenate, Conv2D, Conv2DTranspose, GlobalAveragePooling2D, UpSampling2D, LeakyReLU, ReLU, Add, Multiply, Lambda, Dot, BatchNormalization, Activation, ZeroPadding2D, Cropping2D, Cropping1D
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.initializers import TruncatedNormal, he_normal
import tensorflow.keras.backend as K


def l2normalize(v, eps=1e-12):
    return v / (tf.norm(v) + eps)


class ConvSN2D(tf.keras.layers.Conv2D):

    def __init__(self, filters, kernel_size, power_iterations=1, **kwargs):
        super(ConvSN2D, self).__init__(filters, kernel_size, **kwargs)
        self.power_iterations = power_iterations


    def build(self, input_shape):
        super(ConvSN2D, self).build(input_shape)

        if self.data_format == 'channels_first':
            channel_axis = 1
        else:
            channel_axis = -1

        self.u = self.add_weight(self.name + '_u',
            shape=tuple([1, self.kernel.shape.as_list()[-1]]), 
            initializer=tf.initializers.RandomNormal(0, 1),
            trainable=False
        )

    def compute_spectral_norm(self, W, new_u, W_shape):
        for _ in range(self.power_iterations):

            new_v = l2normalize(tf.matmul(new_u, tf.transpose(W)))
            new_u = l2normalize(tf.matmul(new_v, W))
            
        sigma = tf.matmul(tf.matmul(new_v, W), tf.transpose(new_u))
        W_bar = W/sigma

        with tf.control_dependencies([self.u.assign(new_u)]):
            W_bar = tf.reshape(W_bar, W_shape)

        return W_bar


    def call(self, inputs):
        W_shape = self.kernel.shape.as_list()
        W_reshaped = tf.reshape(self.kernel, (-1, W_shape[-1]))
        new_kernel = self.compute_spectral_norm(W_reshaped, self.u, W_shape)
        outputs = self._convolution_op(inputs, new_kernel)

        if self.use_bias:
            if self.data_format == 'channels_first':
                    outputs = tf.nn.bias_add(outputs, self.bias, data_format='NCHW')
            else:
                outputs = tf.nn.bias_add(outputs, self.bias, data_format='NHWC')
        if self.activation is not None:
            return self.activation(outputs)

        return outputs

class ConvSN2DTranspose(tf.keras.layers.Conv2DTranspose):

    def __init__(self, filters, kernel_size, power_iterations=1, **kwargs):
        super(ConvSN2DTranspose, self).__init__(filters, kernel_size, **kwargs)
        self.power_iterations = power_iterations


    def build(self, input_shape):
        super(ConvSN2DTranspose, self).build(input_shape)

        if self.data_format == 'channels_first':
            channel_axis = 1
        else:
            channel_axis = -1

        self.u = self.add_weight(self.name + '_u',
            shape=tuple([1, self.kernel.shape.as_list()[-1]]), 
            initializer=tf.initializers.RandomNormal(0, 1),
            trainable=False
        )

    def compute_spectral_norm(self, W, new_u, W_shape):
        for _ in range(self.power_iterations):

            new_v = l2normalize(tf.matmul(new_u, tf.transpose(W)))
            new_u = l2normalize(tf.matmul(new_v, W))
            
        sigma = tf.matmul(tf.matmul(new_v, W), tf.transpose(new_u))
        W_bar = W/sigma

        with tf.control_dependencies([self.u.assign(new_u)]):
            W_bar = tf.reshape(W_bar, W_shape)

        return W_bar

    def call(self, inputs):
        W_shape = self.kernel.shape.as_list()
        W_reshaped = tf.reshape(self.kernel, (-1, W_shape[-1]))
        new_kernel = self.compute_spectral_norm(W_reshaped, self.u, W_shape)

        inputs_shape = array_ops.shape(inputs)
        batch_size = inputs_shape[0]
        if self.data_format == 'channels_first':
            h_axis, w_axis = 2, 3
        else:
            h_axis, w_axis = 1, 2

        height, width = inputs_shape[h_axis], inputs_shape[w_axis]
        kernel_h, kernel_w = self.kernel_size
        stride_h, stride_w = self.strides

        if self.output_padding is None:
            out_pad_h = out_pad_w = None
        else:
            out_pad_h, out_pad_w = self.output_padding

        out_height = conv_utils.deconv_output_length(height,
                                                    kernel_h,
                                                    padding=self.padding,
                                                    output_padding=out_pad_h,
                                                    stride=stride_h,
                                                    dilation=self.dilation_rate[0])
        out_width = conv_utils.deconv_output_length(width,
                                                    kernel_w,
                                                    padding=self.padding,
                                                    output_padding=out_pad_w,
                                                    stride=stride_w,
                                                    dilation=self.dilation_rate[1])
        if self.data_format == 'channels_first':
            output_shape = (batch_size, self.filters, out_height, out_width)
        else:
            output_shape = (batch_size, out_height, out_width, self.filters)

        output_shape_tensor = array_ops.stack(output_shape)
        outputs = K.conv2d_transpose(
            inputs,
            new_kernel,
            output_shape_tensor,
            strides=self.strides,
            padding=self.padding,
            data_format=self.data_format,
            dilation_rate=self.dilation_rate)

        if not context.executing_eagerly():
            out_shape = self.compute_output_shape(inputs.shape)
            outputs.set_shape(out_shape)

        if self.use_bias:
            outputs = tf.nn.bias_add(
              outputs,
              self.bias,
              data_format=conv_utils.convert_data_format(self.data_format, ndim=4))

        if self.activation is not None:
            return self.activation(outputs)
        return outputs  

class DenseSN(Dense):
    def build(self, input_shape):
        super(DenseSN, self).build(input_shape)

        self.u = self.add_weight(self.name + '_u',
            shape=tuple([1, self.kernel.shape.as_list()[-1]]), 
            initializer=tf.initializers.RandomNormal(0, 1),
            trainable=False)
        
    def compute_spectral_norm(self, W, new_u, W_shape):
        new_v = l2normalize(tf.matmul(new_u, tf.transpose(W)))
        new_u = l2normalize(tf.matmul(new_v, W))
        sigma = tf.matmul(tf.matmul(new_v, W), tf.transpose(new_u))
        W_bar = W/sigma
        with tf.control_dependencies([self.u.assign(new_u)]):
            W_bar = tf.reshape(W_bar, W_shape)
        return W_bar
        
    def call(self, inputs):
        W_shape = self.kernel.shape.as_list()
        W_reshaped = tf.reshape(self.kernel, (-1, W_shape[-1]))
        new_kernel = self.compute_spectral_norm(W_reshaped, self.u, W_shape)
        rank = len(inputs.shape)
        if rank > 2:
            outputs = standard_ops.tensordot(inputs, new_kernel, [[rank - 1], [0]])
            if not context.executing_eagerly():
                shape = inputs.shape.as_list()
                output_shape = shape[:-1] + [self.units]
                outputs.set_shape(output_shape)
        else:
            inputs = math_ops.cast(inputs, self._compute_dtype)
            if K.is_sparse(inputs):
                outputs = sparse_ops.sparse_tensor_dense_matmul(inputs, new_kernel)
            else:
                outputs = gen_math_ops.mat_mul(inputs, new_kernel)
        if self.use_bias:
            outputs = tf.nn.bias_add(outputs, self.bias)
        if self.activation is not None:
            return self.activation(outputs)
        return outputs


In [12]:
#Networks Architecture

init = tf.keras.initializers.he_uniform()

def conv2d(layer_input, filters, kernel_size=4, strides=2, padding='same', leaky=True, bnorm=True, sn=True):
    if leaky:
        Activ = LeakyReLU(alpha=0.2)
    else:
        Activ = ReLU()
    if sn:
        d = ConvSN2D(filters, kernel_size=kernel_size, strides=strides, padding=padding, kernel_initializer=init, use_bias=False)(layer_input)
    else:
        d = Conv2D(filters, kernel_size=kernel_size, strides=strides, padding=padding, kernel_initializer=init, use_bias=False)(layer_input)
    if bnorm:
        d = BatchNormalization()(d)
    d = Activ(d)
    return d

def deconv2d(layer_input, layer_res, filters, kernel_size=4, conc=True, scalev=False, bnorm=True, up=True, padding='same', strides=2):
    if up:
        u = UpSampling2D((1,2))(layer_input)
        u = ConvSN2D(filters, kernel_size, strides=(1,1), kernel_initializer=init, use_bias=False, padding=padding)(u)
    else:
        u = ConvSN2DTranspose(filters, kernel_size, strides=strides, kernel_initializer=init, use_bias=False, padding=padding)(layer_input)
    if bnorm:
        u = BatchNormalization()(u)
    u = LeakyReLU(alpha=0.2)(u)
    if conc:
        u = Concatenate()([u,layer_res])
    return u

#Extract function: splitting spectrograms
def extract_image(im):
    im1 = Cropping2D(((0,0), (0, 2*(im.shape[2]//3))))(im)
    im2 = Cropping2D(((0,0), (im.shape[2]//3,im.shape[2]//3)))(im)
    im3 = Cropping2D(((0,0), (2*(im.shape[2]//3), 0)))(im)
    return im1,im2,im3

#Assemble function: concatenating spectrograms
def assemble_image(lsim):
    im1,im2,im3 = lsim
    imh = Concatenate(2)([im1,im2,im3])
    return imh

#U-NET style architecture
def build_generator(input_shape):
    h,w,c = input_shape
    inp = Input(shape=input_shape)
    #downscaling
    g0 = tf.keras.layers.ZeroPadding2D((0,1))(inp)
    g1 = conv2d(g0, 256, kernel_size=(h,3), strides=1, padding='valid')
    g2 = conv2d(g1, 256, kernel_size=(1,9), strides=(1,2))
    g3 = conv2d(g2, 256, kernel_size=(1,7), strides=(1,2))
    #upscaling
    g4 = deconv2d(g3,g2, 256, kernel_size=(1,7), strides=(1,2))
    g5 = deconv2d(g4,g1, 256, kernel_size=(1,9), strides=(1,2), bnorm=False)
    g6 = ConvSN2DTranspose(1, kernel_size=(h,1), strides=(1,1), kernel_initializer=init, padding='valid', activation='tanh')(g5)
    return Model(inp,g6, name='G')

#Siamese Network
def build_siamese(input_shape):
    h,w,c = input_shape
    inp = Input(shape=input_shape)
    g1 = conv2d(inp, 256, kernel_size=(h,3), strides=1, padding='valid', sn=False)
    g2 = conv2d(g1, 256, kernel_size=(1,9), strides=(1,2), sn=False)
    g3 = conv2d(g2, 256, kernel_size=(1,7), strides=(1,2), sn=False)
    g4 = Flatten()(g3)
    g5 = Dense(vec_len)(g4)
    return Model(inp, g5, name='S')

#Discriminator (Critic) Network
def build_critic(input_shape):
    h,w,c = input_shape
    inp = Input(shape=input_shape)
    g1 = conv2d(inp, 512, kernel_size=(h,3), strides=1, padding='valid', bnorm=False)
    g2 = conv2d(g1, 512, kernel_size=(1,9), strides=(1,2), bnorm=False)
    g3 = conv2d(g2, 512, kernel_size=(1,7), strides=(1,2), bnorm=False)
    g4 = Flatten()(g3)
    g4 = DenseSN(1, kernel_initializer=init)(g4)
    return Model(inp, g4, name='C')

In [18]:
#Load past models from path to resume training or test
def load(path):
    gen = build_generator((hop,shape,1))
    siam = build_siamese((hop,shape,1))
    critic = build_critic((hop,3*shape,1))
    gen.load_weights(path+'/gen.h5')
    critic.load_weights(path+'/critic.h5')
    siam.load_weights(path+'/siam.h5')
    return gen,critic,siam

#Build models
def build():
    gen = build_generator((hop,shape,1))
    siam = build_siamese((hop,shape,1))
    critic = build_critic((hop,3*shape,1))                                          #the discriminator accepts as input spectrograms of triple the width of those generated by the generator
    return gen,critic,siam

#Generate a random batch to display current training results
def testgena():
    sw = True
    while sw:
        a = np.random.choice(aspec)
        if a.shape[1]//shape!=1:
            sw=False
    dsa = []
    if a.shape[1]//shape>6:
        num=6
    else:
        num=a.shape[1]//shape
    rn = np.random.randint(a.shape[1]-(num*shape))
    for i in range(num):
        im = a[:,rn+(i*shape):rn+(i*shape)+shape]
        im = np.reshape(im, (im.shape[0],im.shape[1],1))
        dsa.append(im)
    return np.array(dsa, dtype=np.float32)

#Show results mid-training
def save_test_image_full(path):
    a = testgena()
    print(a.shape)
    ab = gen(a, training=False)
    ab = testass(ab)
    a = testass(a)
    abwv = deprep(ab)
    awv = deprep(a)
    sf.write(path+'/new_file.wav', abwv, sr)
    IPython.display.display(IPython.display.Audio(np.squeeze(abwv), rate=sr))
    IPython.display.display(IPython.display.Audio(np.squeeze(awv), rate=sr))
    fig, axs = plt.subplots(ncols=2)
    axs[0].imshow(np.flip(a, -2), cmap=None)
    axs[0].axis('off')
    axs[0].set_title('Source')
    axs[1].imshow(np.flip(ab, -2), cmap=None)
    axs[1].axis('off')
    axs[1].set_title('Generated')
    plt.show()

#Save in training loop
def save_end(epoch,gloss,closs,mloss,n_save=3,save_path='/data/output/models'):                 #use custom save_path (i.e. Drive '../content/drive/My Drive/')
    if epoch % n_save == 0:
        print('Saving...')
        path = f'{save_path}/MELGANVC-{str(gloss)[:9]}-{str(closs)[:9]}-{str(mloss)[:9]}'
        os.mkdir(path)
        gen.save_weights(path+'/gen.h5')
        critic.save_weights(path+'/critic.h5')
        siam.save_weights(path+'/siam.h5')
        save_test_image_full(path)
        
#Losses

def mae(x,y):
    return tf.reduce_mean(tf.abs(x-y))

def mse(x,y):
    return tf.reduce_mean((x-y)**2)

def loss_travel(sa,sab,sa1,sab1):
    l1 = tf.reduce_mean(((sa-sa1) - (sab-sab1))**2)
    l2 = tf.reduce_mean(tf.reduce_sum(-(tf.nn.l2_normalize(sa-sa1, axis=[-1]) * tf.nn.l2_normalize(sab-sab1, axis=[-1])), axis=-1))
    return l1+l2

def loss_siamese(sa,sa1):
    logits = tf.sqrt(tf.reduce_sum((sa-sa1)**2, axis=-1, keepdims=True))
    return tf.reduce_mean(tf.square(tf.maximum((delta - logits), 0)))

def d_loss_f(fake):
    return tf.reduce_mean(tf.maximum(1 + fake, 0))

def d_loss_r(real):
    return tf.reduce_mean(tf.maximum(1 - real, 0))

def g_loss_f(fake):
    return tf.reduce_mean(- fake)

#Get models and optimizers
def get_networks(shape, load_model=False, path=None):
    if not load_model:
        gen,critic,siam = build()
    else:
        gen,critic,siam = load(path)

    opt_gen = Adam(0.0001, 0.5)
    opt_disc = Adam(0.0001, 0.5)

    return gen,critic,siam, [opt_gen,opt_disc]

#Set learning rate
def update_lr(lr):
    opt_gen.learning_rate = lr
    opt_disc.learning_rate = lr

#Training Functions

#Train Generator, Siamese and Critic
@tf.function
def train_all(a,b):
    #splitting spectrogram in 3 parts
    aa,aa2,aa3 = extract_image(a) 
    bb,bb2,bb3 = extract_image(b)

    with tf.GradientTape() as tape_gen, tf.GradientTape() as tape_disc:
        #translating A to B
        fab = gen(aa, training=True)
        fab2 = gen(aa2, training=True)
        fab3 = gen(aa3, training=True)
        #identity mapping B to B                                                        COMMENT THESE 3 LINES IF THE IDENTITY LOSS TERM IS NOT NEEDED
        fid = gen(bb, training=True) 
        fid2 = gen(bb2, training=True)
        fid3 = gen(bb3, training=True)
        #concatenate/assemble converted spectrograms
        fabtot = assemble_image([fab,fab2,fab3])
        #feed concatenated spectrograms to critic
        cab = critic(fabtot, training=True)
        cb = critic(b, training=True)
        #feed 2 pairs (A,G(A)) extracted spectrograms to Siamese
        sab = siam(fab, training=True)
        sab2 = siam(fab3, training=True)
        sa = siam(aa, training=True)
        sa2 = siam(aa3, training=True)
        #identity mapping loss
        loss_id = (mae(bb,fid)+mae(bb2,fid2)+mae(bb3,fid3))/3.                         #loss_id = 0. IF THE IDENTITY LOSS TERM IS NOT NEEDED
        #travel loss
        loss_m = loss_travel(sa,sab,sa2,sab2)+loss_siamese(sa,sa2)
        #generator and critic losses
        loss_g = g_loss_f(cab)
        loss_dr = d_loss_r(cb)
        loss_df = d_loss_f(cab)
        loss_d = (loss_dr+loss_df)/2.
        #generator+siamese total loss
        lossgtot = loss_g+10.*loss_m+0.5*loss_id                                       #CHANGE LOSS WEIGHTS HERE  (COMMENT OUT +w*loss_id IF THE IDENTITY LOSS TERM IS NOT NEEDED)
  
    #computing and applying gradients
    grad_gen = tape_gen.gradient(lossgtot, gen.trainable_variables+siam.trainable_variables)
    opt_gen.apply_gradients(zip(grad_gen, gen.trainable_variables+siam.trainable_variables))

    grad_disc = tape_disc.gradient(loss_d, critic.trainable_variables)
    opt_disc.apply_gradients(zip(grad_disc, critic.trainable_variables))

    return loss_dr,loss_df,loss_g,loss_id

#Train Critic only
@tf.function
def train_d(a,b):
    aa,aa2,aa3 = extract_image(a)
    with tf.GradientTape() as tape_disc:

        fab = gen(aa, training=True)
        fab2 = gen(aa2, training=True)
        fab3 = gen(aa3, training=True)
        fabtot = assemble_image([fab,fab2,fab3])

        cab = critic(fabtot, training=True)
        cb = critic(b, training=True)

        loss_dr = d_loss_r(cb)
        loss_df = d_loss_f(cab)

        loss_d = (loss_dr+loss_df)/2.
  
    grad_disc = tape_disc.gradient(loss_d, critic.trainable_variables)
    opt_disc.apply_gradients(zip(grad_disc, critic.trainable_variables))

    return loss_dr,loss_df

def train(epochs, batch_size=16, lr=0.0001, n_save=6, gupt=5, dsa =[], dsb=[], save_path=''):
    import time
    update_lr(lr)
    df_list = []
    dr_list = []
    g_list = []
    id_list = []
    c = 0
    g = 0

    for epoch in range(epochs):
        bef = time.time()
        
        for batchi,(a,b) in enumerate(zip(dsa,dsb)):
          
            if batchi % gupt==0:
                dloss_t,dloss_f,gloss,idloss = train_all(a,b)
            else:
                dloss_t,dloss_f = train_d(a,b)

            df_list.append(dloss_f)
            dr_list.append(dloss_t)
            g_list.append(gloss)
            id_list.append(idloss)
            c += 1
            g += 1

            if batchi%600==0:
                print(f'[Epoch {epoch}/{epochs}] [Batch {batchi}] [D loss f: {np.mean(df_list[-g:], axis=0)} ', end='')
                print(f'r: {np.mean(dr_list[-g:], axis=0)}] ', end='')
                print(f'[G loss: {np.mean(g_list[-g:], axis=0)}] ', end='')
                print(f'[ID loss: {np.mean(id_list[-g:])}] ', end='')
                print(f'[LR: {lr}]')
                g = 0
            nbatch=batchi

        print(f'Completed epoch {epoch}/{epochs} /t Time/Batch {(time.time()-bef)/nbatch}')
        save_end(epoch,np.mean(g_list[-n_save*c:], axis=0),np.mean(df_list[-n_save*c:], axis=0),np.mean(id_list[-n_save*c:], axis=0),n_save=n_save, save_path=save_path)
        print(f'Mean D loss: {np.mean(df_list[-c:], axis=0)} Mean G loss: {np.mean(g_list[-c:], axis=0)} Mean ID loss: {np.mean(id_list[-c:], axis=0)}')
        c = 0

In [14]:
exprDirName="/data/output/models/exp5"

In [15]:
import os

try:
    os.makedirs(exprDirName)    
    print("Directory " , exprDirName ,  " Created ")
except FileExistsError:
    print("Directory " , exprDirName ,  " already exists")  

Directory  /data/output/models/exp5  Created 


In [16]:
gen,critic,siam, [opt_gen,opt_disc] = get_networks(shape, load_model=False, path=exprDirName)

In [19]:
#Training

#n_save = how many epochs between each saving and displaying of results
#gupt = how many discriminator updates for generator+siamese update

train(5, batch_size=bs, lr=0.00001, n_save=10, gupt=3, dsa=dsa, dsb=dsb, save_path=exprDirName)

[Epoch 0/5] [Batch 0] [D loss f: 0.9928476810455322 r: 1.0580847263336182] [G loss: 0.007152269594371319] [ID loss: 0.6795355081558228] [LR: 1e-05]
[Epoch 0/5] [Batch 600] [D loss f: 0.06871896982192993 r: 0.02081494964659214] [G loss: 1.42197847366333] [ID loss: 0.6884912848472595] [LR: 1e-05]
[Epoch 0/5] [Batch 1200] [D loss f: 0.00020646884513553232 r: 3.463685425231233e-05] [G loss: 1.6386712789535522] [ID loss: 0.6647340655326843] [LR: 1e-05]
[Epoch 0/5] [Batch 1800] [D loss f: 0.0002417055075056851 r: 3.729834497789852e-05] [G loss: 1.5521677732467651] [ID loss: 0.6430549621582031] [LR: 1e-05]
[Epoch 0/5] [Batch 2400] [D loss f: 0.00019096990581601858 r: 9.76934825303033e-05] [G loss: 1.52241051197052] [ID loss: 0.6007041335105896] [LR: 1e-05]
[Epoch 0/5] [Batch 3000] [D loss f: 0.000835413346067071 r: 0.0005725978408008814] [G loss: 1.690484881401062] [ID loss: 0.5470640659332275] [LR: 1e-05]
[Epoch 0/5] [Batch 3600] [D loss f: 0.0062784901820123196 r: 0.003390100784599781] [G l

NameError: name 'testass' is not defined